This file takes the original dataset from Kaggle and converts the dataset into Python-readable values

In [22]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import math
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
from tabulate import tabulate
import sys
import copy
sys.path.append('C:\\Users\\chery\\Documents\\SeattlePolice911Response')
from utils.baseML import BaseML
from datetime import datetime
from sklearn import preprocessing

Read in CSV

In [23]:
spir_original = pd.read_csv("Seattle_Police_Department_911_Incident_Response.csv")

C:\Users\chery\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
print(spir_original.columns)

Index(['CAD CDW ID', 'CAD Event Number', 'General Offense Number',
       'Event Clearance Code', 'Event Clearance Description',
       'Event Clearance SubGroup', 'Event Clearance Group',
       'Event Clearance Date', 'Hundred Block Location', 'District/Sector',
       'Zone/Beat', 'Census Tract', 'Longitude', 'Latitude',
       'Incident Location', 'Initial Type Description',
       'Initial Type Subgroup', 'Initial Type Group', 'At Scene Time'],
      dtype='object')


 Drop unused Columns

In [24]:
drop_columns= ['CAD CDW ID', 'CAD Event Number', 'General Offense Number', 'Hundred Block Location', 'District/Sector',
       'Zone/Beat', 'Longitude', 'Latitude',
       'Incident Location']

spir_drop_unused_col = spir_original.drop(labels=drop_columns, axis=1)

Create list of unique census tracts. This is used to pull data from the US Census API 5 year American Community Survey

In [13]:
census_tracts = spir_drop_unused_col["Census Tract"].unique()
census_tracts = pd.DataFrame(census_tracts, columns=["census_tracts"])
census_tracts.dropna(axis =0, inplace=True)
census_tracts.census_tracts=census_tracts.census_tracts.apply(format_tract)
census_tracts.census_tracts = pd.Series(census_tracts.census_tracts.unique())
census_tracts.dropna(axis=0, inplace=True)
census_tracts.to_csv("census_tracts.csv")

Drop rows with missing values

In [25]:
spir_dropna = spir_drop_unused_col.dropna(axis=0, how='any')

Fix date and time

In [93]:
class Seattle_Police_ML(BaseML):

    def dayDifference(self, fdate, ldate):
        delta = ldate - fdate
        return(delta)

    def transform_times(self):
        dt_format = '%m/%d/%Y %I:%M:%S %p'
        #self.df["at_scene_datetime"] = self.df.apply(lambda x: x.At_Scene_Date + "T" + x.At_Scene_Time, axis=1)
        #self.df['at_scene_temp'] = self.df.at_scene_datetime.apply(lambda x: datetime.fromisoformat(x))
        self.df['at_scene_time'] = self.df["At Scene Time"].apply(lambda x: datetime.strptime(x, dt_format))
        #self.df.Event_Clearance_Date.fillna("2021-05-13", inplace=True)
        #self.df.Event_Clearance_Time.fillna("00:00:00", inplace=True)
        
        #self.df[["Event_Clearance_Date", "Event_Clearance_Time"]] = self.df[["Event_Clearance_Date", "Event_Clearance_Time"]].astype(str)
        #print(spir.df.Event_Clearance_Date.dtype)
        
        #self.df["event_clear_datetime"] = self.df.apply(lambda x: x.Event_Clearance_Date + "T" + x.Event_Clearance_Time, axis=1)
        #self.df['event_clear_temp'] = self.df.event_clear_datetime.apply(lambda x: datetime.fromisoformat(x))
        self.df['event_clear_time']=self.df["Event Clearance Date"].apply(lambda x: datetime.strptime(x, dt_format))
        self.df['time_at_scene'] = self.df.apply(lambda x: pd.to_timedelta(self.dayDifference(x["at_scene_time"], x["event_clear_time"])), axis = 1)
        self.df.drop(["At Scene Time", "Event Clearance Date"], axis = 1, inplace = True)
        #self.df.drop(self.df[self.df['Schedule_to_appt_days'] < 0].index, inplace = True)
        #self.df.dropna(inplace=True)
        print(self.df.sample(3))

    def simplify_times(self):
        bins = (
            pd.Timedelta(minutes = 0),
            pd.Timedelta(minutes = 30),
            pd.Timedelta(minutes = 60),
            pd.Timedelta(hours = 4),
            pd.Timedelta(hours = 8),
            pd.Timedelta(hours = 12),
            pd.Timedelta(days = 1),
            pd.Timedelta(days = 36500)
        )

        labels = ['< 30min', '30-60min','1-4hrs', '4-8hrs','8-12hrs','12-24hrs', '24hrs+']

        return pd.cut(self.df['time_at_scene'], bins, labels = labels)

    def map_func(self,parameter_df, x):
        try:
            return parameter_df.loc[math.floor(x["Census Tract"]), str(datetime.strptime(x.at_scene_time, "%Y-%m-%d %H:%M:%S").year)]
        except KeyError:
            return None
            pass
            #print(math.floor(x["Census Tract"]))

    
    def add_census_data(self, census_filename, parameter_name):
        parameter_df = pd.read_csv(census_filename, index_col=1)
        print(self.df["Census Tract"].sample(3))
        self.df[parameter_name]=self.df.apply(lambda x: self.map_func(parameter_df, x), axis=1)
        
        
    def normalize_features(self):
        scaler = preprocessing.StandardScaler().fit(self.df)
        df_scaled = scaler.transform(self.df)
        self.df_normalized = pd.DataFrame(df_scaled, columns = self.df.columns, dtype= 'int64')


In [46]:
year=datetime.today().year
test=pd.DataFrame([[1,2,3],
            ['a','b','c']], columns=[2021, 2012, 2013])
print(test[year])

print(spir_final.df["Census Tract"].sample(3))


0    1
1    a
Name: 2021, dtype: object
1038734    4800.3047
892743     8002.2005
1072815    7700.1029
Name: Census Tract, dtype: float64


In [49]:
test_df= pd.read_csv("white_percent.csv", index_col=1)
print(test_df.loc[math.floor(1200.2000), '2017'])

0.631121868


In [9]:
print(spir_drop_unused_col['Event Clearance Date'].sample(3))


65275      10/01/2010 05:07:00 PM
1066012    03/23/2017 05:38:44 AM
49629      09/12/2010 01:41:00 PM
Name: Event Clearance Date, dtype: object


In [84]:
spir_final = Seattle_Police_ML()
spir_final.df = copy.deepcopy(spir_dropna)
spir_final.transform_times()
spir_final.simplify_times()


         Event Clearance Code Event Clearance Description  \
798918                  280.0           SUSPICIOUS PERSON   
1106900                 245.0          DISTURBANCE, OTHER   
688610                  460.0  TRAFFIC (MOVING) VIOLATION   

         Event Clearance SubGroup     Event Clearance Group  Census Tract  \
798918   SUSPICIOUS CIRCUMSTANCES  SUSPICIOUS CIRCUMSTANCES     7200.1042   
1106900              DISTURBANCES              DISTURBANCES     4600.1001   
688610      TRAFFIC RELATED CALLS     TRAFFIC RELATED CALLS     7300.1003   

                               Initial Type Description  \
798918   WEAPN-IP/JO-GUN,DEADLY WPN (NO THRT/ASLT/DIST)   
1106900                             NUISANCE - MISCHIEF   
688610          TRAFFIC STOP - OFFICER INITIATED ONVIEW   

              Initial Type Subgroup          Initial Type Group  \
798918                WEAPONS CALLS               WEAPONS CALLS   
1106900  MISCELLANEOUS MISDEMEANORS  MISCELLANEOUS MISDEMEANORS   
688610  

255          1-4hrs
2147         1-4hrs
2376         1-4hrs
2433         1-4hrs
2444         1-4hrs
             ...   
1433846      1-4hrs
1433847    30-60min
1433850      1-4hrs
1433851      1-4hrs
1433852    30-60min
Name: time_at_scene, Length: 394768, dtype: category
Categories (7, object): ['< 30min' < '30-60min' < '1-4hrs' < '4-8hrs' < '8-12hrs' < '12-24hrs' < '24hrs+']

In [81]:
spir_final.df.drop(["Initial Type Description", "Initial Type Group"], axis=1, inplace=True)
#spir_final.df = spir_final.df[math.floor(spir_final.df["Census Tract"])!=  1700]

TypeError: cannot convert the series to <class 'float'>

In [85]:
spir_final.df.to_csv("SPIR_transformed_times.csv")

In [94]:
spir_final = Seattle_Police_ML("SPIR_transformed_times.csv")
print(spir_final.df["Census Tract"].sample(3))
spir_final.add_census_data("white_percent.csv","white")


39080      8700.3000
120679    10300.2006
225696     1800.2003
Name: Census Tract, dtype: float64
186021     1200.1002
255802     3200.7001
181707    11400.2009
Name: Census Tract, dtype: float64


In [95]:
spir_final.add_census_data("total_pop_by_tract.csv","total_pop")
spir_final.add_census_data("male_percent.csv","male")
spir_final.add_census_data("notUScitizen_percent.csv","notUScitizen")
spir_final.add_census_data("asian_percent.csv","asian")
spir_final.add_census_data("black_percent.csv","black")
spir_final.add_census_data("native_percent.csv","native")
spir_final.add_census_data("other_race_percent.csv","other_race")
spir_final.add_census_data("two_races_percent.csv","two_races")
spir_final.add_census_data("some_college_percent.csv","some_college")
spir_final.add_census_data("bachelors_percent.csv","bachelors")
spir_final.add_census_data("grad_deg_percent.csv","grad_deg")
spir_final.add_census_data("under18MC_percent.csv","under18MC")
spir_final.add_census_data("under18MS_percent.csv","under18MS")
spir_final.add_census_data("under18FS_percent.csv","under18FS")
spir_final.add_census_data("under18T_percent.csv","under18T")


149903     1702.3011
189698     7900.2008
271683    10900.2092
Name: Census Tract, dtype: float64
71230      4900.1010
359040    11800.5006
370569     8400.2012
Name: Census Tract, dtype: float64
101004      100.3002
190418    11000.5002
330239     1700.3016
Name: Census Tract, dtype: float64
122802     4700.4005
192808    11700.2022
281462     7500.5009
Name: Census Tract, dtype: float64
11269     10600.5012
306098     4500.1014
154745     8200.1002
Name: Census Tract, dtype: float64
278654    7300.3022
380893    8400.1008
118300    8100.3013
Name: Census Tract, dtype: float64
242512     7500.5005
198953     8100.1011
121390    10100.5034
Name: Census Tract, dtype: float64
277903    11400.3000
273491     7500.5006
348087     9800.2001
Name: Census Tract, dtype: float64
62367     11700.2004
176760     8100.1019
2319       6600.1024
Name: Census Tract, dtype: float64
120783    26500.1000
277098     9000.3020
101157     9200.1015
Name: Census Tract, dtype: float64
379660    5802.3025
385

In [98]:
spir_final.df.to_csv("SPIR_census_added.csv")


In [97]:
spir_final.df.dropna(axis=0, inplace=True)

In [99]:
spir_final.df.columns

Index(['Unnamed: 0', 'Event Clearance Code', 'Event Clearance Description',
       'Event Clearance SubGroup', 'Event Clearance Group', 'Census Tract',
       'Initial Type Description', 'Initial Type Subgroup',
       'Initial Type Group', 'at_scene_time', 'event_clear_time',
       'time_at_scene', 'white', 'total_pop', 'male', 'notUScitizen', 'asian',
       'black', 'native', 'other_race', 'two_races', 'some_college',
       'bachelors', 'grad_deg', 'under18MC', 'under18MS', 'under18FS',
       'under18T'],
      dtype='object')

In [100]:
print(spir_final.df['Initial Type Description'].value_counts())

SUSPICIOUS PERSON, VEHICLE OR INCIDENT           28090
THEFT (DOES NOT INCLUDE SHOPLIFT OR SVCS)        27716
DISTURBANCE, MISCELLANEOUS/OTHER                 25488
TRAFFIC STOP - OFFICER INITIATED ONVIEW          19858
BURG - RES (INCL UNOCC STRUCTURES ON PROP)       17027
                                                 ...  
NORAD - INCIDENTS INV AIRCRAFT (EXCL CRASHES)        2
ALARM - EQUIPMENT                                    2
NARCOTICS - VIOLATION OF SODA ORDER                  1
TRU - PURSE SNATCH                                   1
TRU - LURING                                         1
Name: Initial Type Description, Length: 169, dtype: int64


In [103]:
spir_final.df.drop(["Unnamed: 0", "Event Clearance Description", "Event Clearance SubGroup", "Event Clearance Group", "Initial Type Subgroup", "Initial Type Group"], axis=1, inplace=True)

In [104]:
print(spir_final.df.columns)

Index(['Event Clearance Code', 'Census Tract', 'Initial Type Description',
       'at_scene_time', 'event_clear_time', 'time_at_scene', 'white',
       'total_pop', 'male', 'notUScitizen', 'asian', 'black', 'native',
       'other_race', 'two_races', 'some_college', 'bachelors', 'grad_deg',
       'under18MC', 'under18MS', 'under18FS', 'under18T'],
      dtype='object')


In [105]:
spir_final.df['Initial Type Description'] = spir_final.df['Initial Type Description'].astype('category')
print(spir_final.df.dtypes)

Event Clearance Code         float64
Census Tract                 float64
Initial Type Description    category
at_scene_time                 object
event_clear_time              object
time_at_scene                 object
white                        float64
total_pop                    float64
male                         float64
notUScitizen                 float64
asian                        float64
black                        float64
native                       float64
other_race                   float64
two_races                    float64
some_college                 float64
bachelors                    float64
grad_deg                     float64
under18MC                    float64
under18MS                    float64
under18FS                    float64
under18T                     float64
dtype: object


In [106]:
spir_final.df["initial_type_desc_cat"] = spir_final.df["Initial Type Description"].cat.codes
print(spir_final.df.head())

   Event Clearance Code  Census Tract  \
0                 161.0     6000.1011   
1                  50.0    10300.2005   
2                 430.0     2800.1003   
3                  50.0     3400.3000   
4                 430.0     7900.5006   

                            Initial Type Description        at_scene_time  \
0                                NUISANCE - MISCHIEF  2016-01-24 10:46:45   
1         BURG - RES (INCL UNOCC STRUCTURES ON PROP)  2016-01-24 10:55:44   
2  MOTOR VEHICLE COLLISION WITH INJURIES - PRIORI...  2016-01-24 10:17:27   
3         BURG - RES (INCL UNOCC STRUCTURES ON PROP)  2016-01-24 10:29:37   
4                      DUI - DRIVING UNDER INFLUENCE  2016-01-24 08:58:44   

      event_clear_time    time_at_scene     white  total_pop      male  \
0  2016-01-24 11:59:45  0 days 01:13:00  0.839407     5598.0  0.471240   
1  2016-01-24 12:01:15  0 days 01:05:31  0.457715     6397.0  0.494607   
2  2016-01-24 12:06:53  0 days 01:49:26  0.910944     4907.0  0.4787

In [111]:
spir_final.df.at_scene_time = spir_final.df.at_scene_time.astype('datetime64')
spir_final.df.event_clear_time = spir_final.df.event_clear_time.astype('datetime64')
spir_final.df.time_at_scene = pd.to_timedelta(spir_final.df.time_at_scene)

print(spir_final.df.time_at_scene.dtypes)

timedelta64[ns]


In [112]:
print(spir_final.df.head())

   Event Clearance Code  Census Tract  \
0                 161.0     6000.1011   
1                  50.0    10300.2005   
2                 430.0     2800.1003   
3                  50.0     3400.3000   
4                 430.0     7900.5006   

                            Initial Type Description       at_scene_time  \
0                                NUISANCE - MISCHIEF 2016-01-24 10:46:45   
1         BURG - RES (INCL UNOCC STRUCTURES ON PROP) 2016-01-24 10:55:44   
2  MOTOR VEHICLE COLLISION WITH INJURIES - PRIORI... 2016-01-24 10:17:27   
3         BURG - RES (INCL UNOCC STRUCTURES ON PROP) 2016-01-24 10:29:37   
4                      DUI - DRIVING UNDER INFLUENCE 2016-01-24 08:58:44   

     event_clear_time   time_at_scene     white  total_pop      male  \
0 2016-01-24 11:59:45 0 days 01:13:00  0.839407     5598.0  0.471240   
1 2016-01-24 12:01:15 0 days 01:05:31  0.457715     6397.0  0.494607   
2 2016-01-24 12:06:53 0 days 01:49:26  0.910944     4907.0  0.478704   
3 2016-0

In [124]:
noon = datetime(2021, 11, 12,hour = 12)
spir_final.df["at_scene_time_pm"] = spir_final.df.at_scene_time.apply(lambda x: 0 if x.hour < noon.hour else 1)
spir_final.df["event_clear_time_pm"] = spir_final.df.event_clear_time.apply(lambda x: 0 if x.hour < noon.hour else 1)


In [128]:
spir_final.df["at_scene_time_weekday"] = spir_final.df.at_scene_time.apply(datetime.weekday)
spir_final.df["event_clear_time_weekday"] = spir_final.df.event_clear_time.apply(datetime.weekday)

In [130]:
spir_final.df["at_scene_time_month"] = spir_final.df.at_scene_time.apply(lambda x: x.month)
spir_final.df["event_clear_time_month"] = spir_final.df.event_clear_time.apply(lambda x: x.month)

In [131]:
spir_final.df["at_scene_time_year"] = spir_final.df.at_scene_time.apply(lambda x: x.year)
spir_final.df["event_clear_time_year"] = spir_final.df.event_clear_time.apply(lambda x: x.year)

In [135]:
spir_final.df.sample(10)

,Event Clearance Code,Census Tract,Initial Type Description,at_scene_time,event_clear_time,time_at_scene,white,total_pop,male,notUScitizen,...,initial_type_desc_cat,at_scene_time_pm,event_clear_time_pm,at_scene_time_weekday,event_clear_time_weekday,at_scene_time_month,event_clear_time_month,at_scene_time_year,event_clear_time_year,time_at_scene_seconds
7004,245.0,4700.3020,"DISTURBANCE, MISCELLANEOUS/OTHER",2016-02-26 00:02:05,2016-02-26 01:22:38,0 days 01:20:33,0.835542,8677.0,0.498905,0.028927,...,45,0,0,4,4,2,2,2016,2016,4833.0
69304,245.0,5400.2012,"DISTURBANCE, MISCELLANEOUS/OTHER",2016-09-16 19:26:40,2016-09-16 20:17:27,0 days 00:50:47,0.815390,5406.0,0.487976,0.058084,...,45,1,1,4,4,9,9,2016,2016,3047.0
125427,245.0,8500.1010,FIGHT - VERBAL/ORAL (NO WEAPONS),2015-08-11 09:16:00,2015-08-11 11:18:00,0 days 02:02:00,0.520761,4311.0,0.651125,0.116446,...,52,0,0,1,1,8,8,2015,2015,7320.0
12479,430.0,5100.3005,"MOTOR VEHICLE COLLISION, BLOCKING",2016-12-17 11:37:57,2016-12-17 12:06:25,0 days 00:28:28,0.860676,3639.0,0.507282,0.051113,...,80,0,1,5,5,12,12,2016,2016,1708.0
162328,250.0,8100.3003,SUSPICIOUS STOP - OFFICER INITIATED ONVIEW,2016-01-01 16:05:33,2016-01-01 20:05:09,0 days 03:59:36,0.674712,4599.0,0.557513,0.133942,...,125,1,1,4,4,1,1,2016,2016,14376.0
363190,63.0,8300.2017,SECONDARY - THEFT (NOT SHOPLIFT OR SERVICES),2015-04-24 20:24:00,2015-04-24 22:01:00,0 days 01:37:00,0.795633,2794.0,0.544739,0.094846,...,116,1,1,4,4,4,4,2015,2015,5820.0
206572,63.0,4700.4029,"SUSPICIOUS PERSON, VEHICLE OR INCIDENT",2017-04-08 02:26:02,2017-04-08 05:33:30,0 days 03:07:28,0.797896,8649.0,0.494046,0.024627,...,124,0,0,5,5,4,4,2017,2017,11248.0
259261,41.0,7100.1004,THREATS (INCLS IN-PERSON/BY PHONE/IN WRITING),2016-11-08 09:27:49,2016-11-08 11:00:04,0 days 01:32:15,0.714183,4188.0,0.538682,0.210840,...,128,0,0,1,1,11,11,2016,2016,5535.0
384211,130.0,6800.1012,PROPERTY - DAMAGE,2015-07-04 03:57:00,2015-07-04 05:34:00,0 days 01:37:00,0.866188,3206.0,0.513724,0.071740,...,101,0,0,5,5,7,7,2015,2015,5820.0
173189,470.0,5700.1012,PARKING VIOLATION (EXCEPT ABANDONED CAR),2012-09-01 11:21:00,2012-09-01 11:41:00,0 days 00:20:00,0.866763,6252.0,0.468650,0.049264,...,95,0,0,5,5,9,9,2012,2012,1200.0


In [134]:
from datetime import timedelta
spir_final.df["time_at_scene_seconds"] = spir_final.df.time_at_scene.apply(timedelta.total_seconds)

In [136]:
spir_final.df.to_csv("SPIR_encoded_2021112.csv")

In [122]:
today = datetime.today()
noon = datetime(2021, 11, 12,hour = 12)
print(today.hour)
if today.hour < noon.hour:
    print('am')
else:
    print('pm')

16
pm


In [129]:
print(today.month)

11


Isolate missing values